# Start Interactive Dask Cluster

In [ ]:
pip install --upgrade dask-cloudprovider

In [ ]:
pip install --upgrade azureml-sdk

**RESTART YOUR KERNEL**

## Workspace

Get the Azure ML workspace.

In [1]:
from azureml.core import Workspace

ws = Workspace.from_config()
ws

Workspace.create(name='AzureML', subscription_id='6560575d-fa06-4e7d-95fb-f962e74efd7a', resource_group='cody-eastus-rg')

### Start cluster

In [2]:
from dask_cloudprovider import AzureMLCluster

vm_size = 'STANDARD_DS13_V2'
timeout = 7200 

cluster = AzureMLCluster(ws, vm_size=vm_size, scheduler_idle_timeout=timeout, jupyter=True, show_output=True)

################## Creating new compute targe: dask-ct-186009 ##################
Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
############################## Setting up cluster ##############################
WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.
WARNING - If 'arguments' has been provided here and arguments have been specified in 'run_config', 'arguments' provided in ScriptRunConfig initialization will take precedence.
####################### Waiting for scheduler node's IP ########################
........................................................

########################### Scheduler: 10.0.0.4:8786 ###########################
############################# Not on the same VNET #############################
###################### Running in compute instance? False #########

In [3]:
cluster.run

Experiment,Id,Type,Status,Details Page,Docs Page
dask-cloudprovider,dask-cloudprovider_1598986900_3f78a326,azureml.scriptrun,Running,Link to Azure Machine Learning studio,Link to Documentation


In [4]:
cluster.scale(20) # need more than default quota for this 

WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.


In [5]:
# print out cluster widget 
cluster

In [6]:
from dask.distributed import Client 

c = Client(cluster)
c

Client Scheduler: tcp://localhost:9002 Dashboard: http://localhost:9001,Cluster Workers: 20 Cores: 160 Memory: 1.18 TB


In [7]:
#cluster.close()

In [8]:
help(AzureMLCluster)

Help on class AzureMLCluster in module dask_cloudprovider.providers.azure.azureml:

class AzureMLCluster(distributed.deploy.cluster.Cluster)
 |  AzureMLCluster(workspace, compute_target=None, environment_definition=None, experiment_name=None, initial_node_count=None, jupyter=None, jupyter_port=None, dashboard_port=None, scheduler_port=None, scheduler_idle_timeout=None, worker_death_timeout=None, additional_ports=None, admin_username=None, admin_ssh_key=None, datastores=None, code_store=None, vnet_resource_group=None, vnet=None, subnet=None, show_output=False, telemetry_opt_out=None, asynchronous=False, **kwargs)
 |  
 |  Deploy a Dask cluster using Azure ML
 |  
 |  This creates a dask scheduler and workers on an Azure ML Compute Target.
 |  
 |  Parameters
 |  ----------
 |  workspace: azureml.core.Workspace (required)
 |      Azure ML Workspace - see https://aka.ms/azureml/workspace.
 |  
 |  vm_size: str (optional)
 |      Azure VM size to be used in the Compute Target - see https:/